In [65]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import numpy as np
from scipy import *
from scipy import io
from scipy import linalg
from scipy import sparse
import scipy.sparse.linalg
import matplotlib
import matplotlib.pylab as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/test-matrix/A.mm


# The aim of this code is to solve $\mathbf{A}\mathbf{x}=\mathbf{b}$ for $\mathbf{x}$ using the *conjugate gradient* method.

**Reading the matrix and creating the right-hand-side vector:**

In [66]:
A = io.mmread("/kaggle/input/test-matrix/A.mm") #here we read a matrix.
b = np.ones( A.shape[0] ) #here we initialize a rhs vector with ones

In [67]:
print("shape of A", A.shape)
print("norm A = ", sparse.linalg.norm(A))
print("norm b = ", linalg.norm(b))

shape of A (900, 900)
norm A =  253.8582281510686
norm b =  30.0


**The reference result is obtained by the available conjugate gradient solver implemented within scipy:**

In [68]:
[xreference,err] = sparse.linalg.cg(A,b, tol=1e-9)

**Initial guess is obtained as $\mathbf{x}_0 = \mathbf{D}^{-1} \mathbf{b}$, where $\mathbf{D}=\text{diag}(\mathbf{A})$:**

In [69]:
x = np.ones( b.shape[0] )
A_csr = A.tocsr()
for i, element in enumerate(b):
    x[i] = (1.0/A_csr[i,i])*b[i]

**Initializing the residual as $\mathbf{r} = \mathbf{b} - \mathbf{A}\mathbf{x}$:**

In [70]:
r = b.copy()
r -= A*x

**The search direction is initialized as $\mathbf{d} = \mathbf{r}$:**

In [71]:
d = r.copy()

In [72]:
error = linalg.norm(r)
print("error for the initial guess is ", error)

error for the initial guess is  28.78041869049163


**The loop of iterations:**

In [73]:
tolerance = 1e-9
maximum_iterations = 10000
write_steps = True

In [74]:
count = 0
while ( (count < maximum_iterations) and (error > tolerance) ):
    Ad = A*d
    dTAd = 0
    #for i, element in enumerate(Ad):
    #    dTAd += d[i]*element
    dTAd = np.dot(d,Ad)
    alpha = error*error/dTAd
    x += alpha*d
    r -= alpha*Ad
    rTrnew = 0
    errornew = linalg.norm(r)
    beta = errornew*errornew/error/error
    d = beta*d
    d += r
    error = errornew
    if (write_steps):
        print("iteration no.", count, "\t error", error)
    count += 1

iteration no. 0 	 error 61.8949352655528
iteration no. 1 	 error 72.99006912457085
iteration no. 2 	 error 55.431278653791
iteration no. 3 	 error 51.11464576023864
iteration no. 4 	 error 48.1202410004486
iteration no. 5 	 error 38.32759362454813
iteration no. 6 	 error 35.88431456456216
iteration no. 7 	 error 29.220691881921002
iteration no. 8 	 error 23.995263444800695
iteration no. 9 	 error 20.308464924901624
iteration no. 10 	 error 14.71726787638889
iteration no. 11 	 error 10.90464946426536
iteration no. 12 	 error 6.53283612400792
iteration no. 13 	 error 1.7952490778295265
iteration no. 14 	 error 2.0939284317544606
iteration no. 15 	 error 1.3444610331399065
iteration no. 16 	 error 0.9084289732728242
iteration no. 17 	 error 0.5712860504328
iteration no. 18 	 error 0.40296628965813536
iteration no. 19 	 error 0.2643189794547935
iteration no. 20 	 error 0.17099779479990815
iteration no. 21 	 error 0.10838894023166017
iteration no. 22 	 error 0.0574856484081129
iteration no.

**Verification:**

In [75]:
LHS = A*x
print("calculated error", linalg.norm(LHS-b))
LHS = A*xreference
print("reference error", linalg.norm(LHS-b))

calculated error 6.338293654912467e-10
reference error 2.006198253388821e-08


In [76]:
print("norm dx =", linalg.norm(x-xreference))

norm dx = 4.860914324094664e-09
